<a href="https://colab.research.google.com/github/Servetvrll/customer-segmentation-rfm/blob/main/Customer_segmentation_son.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri projem için içeri aktarıyorum.
# numpy: Sayısal işlemler için.
# pandas: Veri manipülasyonu ve analizi için.
# matplotlib.pyplot: Statik, interaktif ve animasyonlu görselleştirmeler için.
# seaborn: Matplotlib üzerine kurulu, daha çekici istatistiksel grafikler oluşturmak için.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Veri setimi Google Drive'ımdan yüklüyorum.
# Colab ortamında çalıştığım için dosya yolunu bu şekilde belirtiyorum.
# Veri setine erişmek için kaggle linki "https://www.kaggle.com/datasets/arjunbhasin2013/ccdata"
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Customer_Segmentation/custemer_segmentation.csv")

# Veri setime ilk bakışı atıyorum:
# İlk 5 satırı görüntüleyerek verinin genel yapısı hakkında fikir ediniyorum.
print("Veri setimin ilk 5 satırı:")
print(data.head())
print("-----------------------------------------")

# Son 5 satırı görüntüleyerek verinin sonuna doğru bir kontrol yapıyorum.
print("Veri setimin son 5 satırı:")
print(data.tail())
print("-----------------------------------------")

# Veri tiplerini ve her sütundaki boş (null) olmayan değer sayısını inceliyorum.
# Bu, eksik veri olup olmadığını ve sütunların doğru veri tipinde olup olmadığını anlamamı sağlıyor.
print("Veri tipleri ve boş (non-null) değer sayıları:")
print(data.info())
print("-----------------------------------------")

# Sayısal sütunların temel istatistiksel özetlerini (ortalama, standart sapma, min, max vb.) görüntülüyorum.
# Bu, verinin dağılımı ve olası aykırı değerler hakkında ilk ipuçlarını veriyor.
print("Sayısal sütunlarımın istatistiksel özeti:")

print(data.describe())
print("-----------------------------------------")

# Her sütundaki toplam boş (NaN) değer sayısını kontrol ediyorum.
# Bu, veri temizleme adımlarım için hangi sütunlara odaklanmam gerektiğini gösteriyor.
print("Her sütundaki toplam boş (NaN) değer sayısı:")
print(data.isnull().sum())
print("-----------------------------------------")


In [ ]:
# 'CUST_ID' sütununu veri setimden çıkarıyorum.
# Bu sütun, müşteri kimlik bilgisini içerdiği için kümeleme analizi için bir özellik olarak kullanılmaz ve gereksizdir.
data.drop(["CUST_ID"], axis=1, inplace=True)
print("CUST_ID sütunu silindikten sonra veri setimin ilk 5 satırı:")
print(data.head())

# Eksik değerleri doldurma (imputation) ve silme işlemleri:
# 'MINIMUM_PAYMENTS' sütunundaki boş değerleri 0 ile dolduruyorum.
# Müşterinin minimum ödeme yapması gerekmediği veya hiç ödeme oluşmadığı durumları temsil edebileceğini varsaydım.
data["MINIMUM_PAYMENTS"] = data["MINIMUM_PAYMENTS"].fillna(0)

# 'CREDIT_LIMIT' sütununda sadece tek bir boş değer olduğunu tespit ettim.
# Bu tek satırı silmek, veri kaybını minimumda tutarken veri setimi temizlememi sağlıyor.
data.dropna(subset=['CREDIT_LIMIT'], inplace=True)
print("\nBoş değerler temizlendikten sonraki durum (tekrar kontrol):")
print(data.isnull().sum())


In [ ]:
# Kümeleme için kullanacağım sütunları belirliyorum.
# Bu sütunlar, müşteri davranışlarını ve finansal özelliklerini yansıtan önemli değişkenlerdir.
columns =['BALANCE',  'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE','ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS'
       ]
df_new = data[columns]

# Veri setimdeki sayısal değişkenlerin dağılımını histogramlarla görselleştiriyorum.
# Bu grafikler, her bir değişkenin nasıl dağıldığını, çarpık olup olmadığını ve olası aykırı değerleri görmemi sağlıyor.
data.hist(figsize=(20, 15), bins=50)
plt.suptitle('Değişkenlerin Dağılımı', fontsize=20) # Ana başlık ekliyorum
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Başlığın grafiklerle çakışmaması için düzenleme
plt.show()

# Değişkenler arasındaki korelasyonu bir ısı haritası (heatmap) ile inceliyorum.
# Bu, hangi değişkenlerin birbiriyle güçlü bir ilişki içinde olduğunu görmemi sağlıyor.
corr_matrix = data.corr()
plt.figure(figsize=(18, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Değişkenler Arasındaki Korelasyon Matrisi')
plt.show()

# Aykırı değerleri (outliers) tespit etmek için her bir sütun için kutu grafikleri çiziyorum.
# Kutu grafikleri, verinin çeyrekler arası dağılımını ve kutunun dışındaki aykırı noktaları gösterir.
plt.figure(figsize=(15,60)) # Büyük bir figür boyutu, tüm grafikleri sığdırmak için
for i,col in enumerate(df_new.columns,1):
  plt.subplot(12, 2, i) # 12 satır, 2 sütunlu bir ızgarada her grafiği yerleştiriyorum
  sns.boxplot(x=data[col])
  plt.title(f'{col} İçin Kutu Grafiği')
plt.tight_layout() # Grafikler arasında boşluk bırakarak çakışmayı önlüyorum
plt.show()

In [ ]:
# Aykırı değerleri IQR (Interquartile Range) yöntemi ile kırpıyorum (capping).
# Bu işlem, aykırı değerleri silmek yerine, onları belirli bir üst veya alt sınıra sabitler.
# Böylece veri kaybı yaşamadan modelin aykırı değerlerden olumsuz etkilenmesini önlüyorum.
print("Aykırı Değerleri Kırpma İşlemi (IQR Yöntemiyle):")
for col in df_new:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower_bound, upper_bound) # Değerleri belirlenen aralıkta tutuyorum
    print(f"'{col}' sütunundaki aykırı değerler [{lower_bound:.2f}, {upper_bound:.2f}] aralığına kırpıldı.")
print("---------------------")

# Aykırı değerleri kırptıktan sonra kutu grafiklerini tekrar çizerek değişimi gözlemliyorum.
# Bu, kırpma işleminin ne kadar etkili olduğunu görsel olarak doğrulamamı sağlıyor.
plt.figure(figsize=(15,60))
for i,col in enumerate(df_new.columns,1):
  plt.subplot(12, 2, i)
  sns.boxplot(x=data[col])
  plt.title(f'{col} İçin Kutu Grafiği (Kırpma Sonrası)')
plt.tight_layout()
plt.show()


In [ ]:
# Bu kısmı kodlarken Gemini'den yardımm aldım.
# Yüksek korelasyonlu değişkenleri tespit etmek için sayısal verilerimi seçiyorum.
numeric_data = data.select_dtypes(include=[np.number])
corr_matrix = numeric_data.corr().abs() # Mutlak değer alarak hem pozitif hem negatif güçlü ilişkileri yakalıyorum.

# Korelasyonu 0.80 ve üzeri olan sütunları görselleştirmek için bir maske oluşturuyorum.
# Bu maske, ısı haritasında sadece güçlü ilişkileri göstererek görseli sadeleştiriyor.
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Matrisin üst üçgenini gizliyorum
mask = mask | (corr_matrix < 0.80) # 0.80'den küçük korelasyonları da gizliyorum

plt.figure(figsize=(18, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=mask)
plt.title('Korelasyonu 0.80 ve Üzeri Olan Sütunlar')
plt.show()

# Veri setimdeki mevcut sütunları görüntülüyorum.
print("Veri setimdeki son sütunlar:")
print(data.columns)


In [ ]:
# Yüksek korelasyonlu sütunları veri setimden çıkarıyorum.
# Bu kararı, iş mantığıma ve değişkenlerin bilgi tekrarına dayanarak verdim.
# 'MINIMUM_PAYMENTS', 'PURCHASES_TRX', 'CASH_ADVANCE_TRX' gibi sütunlar,
# zaten diğer benzer sütunlarla (örn. BALANCE, PURCHASES, CASH_ADVANCE) yüksek korelasyona sahipti.
df = data.drop(['MINIMUM_PAYMENTS', 'PURCHASES_TRX', 'CASH_ADVANCE_TRX'], axis=1)
print("Yüksek korelasyonlu sütunlar silindikten sonra kalan sütunlar:")
print(df.columns)

In [ ]:
# K-Means algoritması için veriyi Standardizasyon (Ölçekleme) işlemi yapıyorum.
# K-Means, mesafeye dayalı bir algoritma olduğu için, tüm değişkenlerin aynı ölçekte olması hayati önem taşır.
# StandardScaler, veriyi ortalaması 0 ve standart sapması 1 olacak şekilde dönüştürüyor.
from sklearn.preprocessing import StandardScaler
X = df.copy() # Kümeleme yapacağım veriyi kopyalıyorum
scaler = StandardScaler() # Ölçekleyici objemi oluşturuyorum
X_scaled = scaler.fit_transform(X) # Veriyi ölçekliyorum
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns) # Ölçeklenmiş veriyi DataFrame'e dönüştürüyorum


In [ ]:
# Optimum küme sayısını (k) bulmak için Dirsek Yöntemini (Elbow Method) kullanıyorum.
# WCSS (Küme İçi Kareler Toplamı), k arttıkça azalır; grafiğin "dirsek" yaptığı nokta ideal k'yi gösterir.
from sklearn.cluster import KMeans
wcss = [] # WCSS değerlerini saklamak için boş bir liste
k_values = range(1, 11) # Deneyeceğim k değerleri aralığı

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_scaled_df) # Ölçeklenmiş veriye modeli uyguluyorum
    wcss.append(kmeans.inertia_) # Modelin inertia_ özelliği WCSS değerini verir

# Dirsek grafiğini çizdiriyorum.
plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss, marker='o', linestyle='--')
plt.title('Dirsek Yöntemi (Elbow Method)')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Küme İçi Kareler Toplamı (WCSS)')
plt.xticks(k_values) # x ekseni etiketlerini k değerleriyle eşleştiriyorum
plt.grid(True)
plt.show()


In [ ]:
# Dirsek Yöntemi net bir sonuç vermediği için Silüet Skorunu deniyorum.
# Silüet Skoru, kümelerin ne kadar iyi ayrıldığını ve veri noktalarının kendi kümelerine ne kadar uyduğunu ölçer.
from sklearn.metrics import silhouette_score
silhouette_scores = [] # Silüet skorlarını saklamak için boş bir liste
k_values = range(2, 11) # k=1 için Silüet Skoru hesaplanamaz, bu yüzden 2'den başlıyorum

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled_df) # Kümeleri tahmin ediyorum
    score = silhouette_score(X_scaled_df, cluster_labels) # Skoru hesaplıyorum
    silhouette_scores.append(score)

# Silüet skorlarının grafiğini çizdiriyorum.
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, marker='o', linestyle='--')
plt.title('Silüet Skoru Yöntemi')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Ortalama Silüet Skoru')
plt.xticks(k_values)
plt.grid(True)
plt.show()

# En yüksek Silüet skorunu veren k değerini buluyorum.
optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
print(f"En yüksek Silüet skorunu veren k değeri: {optimal_k}")
print(f"En yüksek Silüet Skoru: {max(silhouette_scores):.2f}")


In [ ]:
# ÖNEMLİ KARAR NOKTASI: K-Means Modeli Oluşturma
# Dirsek grafiği belirgin bir dirsek noktası sunmasa da, k=4'ün mantıklı bir ayrım olabileceğini gözlemledim.
# Silüet skoru grafiği ise en yüksek değeri k=2 için gösterdi.
# Ancak k=2 gibi çok az sayıda küme, müşterileri "iyi" ve "kötü" gibi çok genel gruplara ayırarak
# pazarlama stratejileri için yeterli çeşitliliği sunmuyordu.
# Bu nedenle, Dirsek Yöntemi'nin işaret ettiği ve iş mantığıma daha uygun olan k=4'ü optimum küme sayısı olarak seçtim.
# Daha yüksek bir küme sayısı olan k=7'yi tercih etmedim çünkü bu, segmentlerin çok fazla alt gruba ayrılmasına
# ve bazı segmentlerin çok az sayıda müşteriden oluşmasına neden oluyordu.
# Bu durum, her bir segment için anlamlı ve uygulanabilir bir pazarlama stratejisi belirlemeyi zorlaştıracaktı.
from sklearn.cluster import KMeans
kmeans_4 = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=42)
df['Segment_4'] = kmeans_4.fit_predict(X_scaled_df) # Müşterileri 4 segmente ayırıyorum

# Her segmentteki müşteri sayısını görüntülüyorum.
print("K=4 için Segment Dağılımı:")
print(df['Segment_4'].value_counts())

# Segmentlerin ortalama değerlerini inceleyerek her bir segmentin profilini çıkarıyorum.
segment_profiles_4 = df.groupby('Segment_4').mean()
print("\nK=4 için Segment Profilleri (Ortalama Değerler):")
print(segment_profiles_4)

In [ ]:
# Sayısal segment etiketlerini anlamlı isimlere dönüştürüyorum.
df['Segment_Name_loc'] = ''
df.loc[df['Segment_4'] == 0, 'Segment_Name_loc'] = 'Cash_Advance_Users'
df.loc[df['Segment_4'] == 1, 'Segment_Name_loc'] = 'Passive_Users'
df.loc[df['Segment_4'] == 2, 'Segment_Name_loc'] = 'Installment_Shoppers'
df.loc[df['Segment_4'] == 3, 'Segment_Name_loc'] = 'Affluent_High_Spenders'

print("\nSegment İsimlerine göre müşteri sayısı (doğrulama):")
print(df['Segment_Name_loc'].value_counts())
print("\nYeni DataFrame'imin ilk 5 satırı (Segment_Name_loc sütunu ile):")
print(df["Segment_Name_loc"].head())

# Orijinal sayısal segment sütununu ('Segment_4') siliyorum.
# Artık daha anlamlı olan 'Segment_Name_loc' sütunum var ve veri tekrarını önlüyorum.
df.drop("Segment_4", axis=1, inplace=True)

In [ ]:
# Kümeleme öncesi ve sonrası veri dağılımını görselleştiriyorum.
# Bu, K-Means algoritmasının veriyi nasıl anlamlı gruplara ayırdığını gözlemlememi sağlıyor.
var1 = 'PURCHASES'
var2 = 'CREDIT_LIMIT'

# Kümeleme öncesi saçılım grafiği
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df[var1], y=df[var2], alpha=0.6)
plt.title('Kümeleme Öncesi Veri Dağılımı')
plt.xlabel(var1)
plt.ylabel(var2)
plt.grid(True)
plt.show()


In [ ]:
# Kümeleme sonrası saçılım grafiği (segmentlere göre renklendirilmiş)
plt.figure(figsize=(12, 8))
sns.scatterplot(x=df[var1], y=df[var2], hue='Segment_Name_loc', palette='viridis', data=df)
plt.title('Kümeleme Sonrası Veri Dağılımı (Segmentlere Göre)')
plt.xlabel(var1)
plt.ylabel(var2)
plt.legend(title='Segmentler')
plt.grid(True)
plt.show()

In [ ]:
# Burada Gemini'den yardım aldım.
# Radar grafiği ile müşteri segmentlerinin profillerini detaylı inceliyorum.
# Bu görselleştirme, her segmentin özelliklerini tek bir grafikte karşılaştırmamı sağlıyor ve
# segmentler arasındaki farkları net bir şekilde ortaya koyuyor.
# Seaborn'dan daha estetik bir stil ve renk paleti kullanıyorum.
plt.style.use('seaborn-v0_8-whitegrid')
segment_profiles_4 = df.groupby('Segment_Name_loc').mean()
features = ['BALANCE', 'PURCHASES', 'CASH_ADVANCE', 'CREDIT_LIMIT', 'PAYMENTS', 'TENURE', 'PRC_FULL_PAYMENT']
profile_df = segment_profiles_4[features]
normalized_profile_df = (profile_df - profile_df.min()) / (profile_df.max() - profile_df.min())

palette = sns.color_palette("Set2", len(normalized_profile_df))
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

angles = np.linspace(0, 2 * np.pi, len(features), endpoint=False).tolist()
angles += angles[:1]
features += features[:1]

for i in range(len(normalized_profile_df)):
    values = normalized_profile_df.iloc[i].values.tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2.5, linestyle='solid', label=normalized_profile_df.index[i], color=palette[i])
    ax.fill(angles, values, alpha=0.3, color=palette[i])

ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_rlabel_position(0)
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(features[:-1], fontsize=14, fontweight='bold')
ax.set_title('Müşteri Segmentlerinin Detaylı Profili', size=20, y=1.1, fontweight='bold')
ax.legend(loc='lower right', bbox_to_anchor=(0.1, -0.1), fontsize=12, ncol=1, frameon=False)

for spine in ax.spines.values():
    spine.set_edgecolor('lightgray')
    spine.set_linewidth(0.5)

plt.show()